In [ ]:
#@title Copyright 2020 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Setup

In [ ]:
# Import the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# Import the folium library; it is used for interactive map display.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add Earth Engine layer drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Image Visualization

Image tiles generated for visualization of raster data defaults to assigning the first three bands to red, green, and blue, respectively. The default stretch is based on the type of data in the band (e.g. floats are stretched in [0,1], 16-bit data are stretched to the full range of possible values), which may or may not be suitable. To achieve desirable visualization effects, you can provide visualization parameters to the `ee.Image.visualize()` function or to the `vis_params` parameter of the `add_ee_layer()` method added to the `folium` mapping library. Specifically, the parameters are:

|Parameter|Description|Type|
|--- |--- |--- |
|`bands`|Comma-delimited list of three band names to be mapped to RGB|list|
|`min`|Value(s) to map to 0|number or list of three numbers, one for each band|
|`max`|Value(s) to map to 255|number or list of three numbers, one for each band|
|`gain`|Value(s) by which to multiply each pixel value|number or list of three numbers, one for each band|
|`bias`|Value(s) to add to each DN|number or list of three numbers, one for each band|
|`gamma`|Gamma correction factor(s)|number or list of three numbers, one for each band|
|`palette`|List of CSS-style color strings (single-band images only)|comma-separated list of hex strings|
|`opacity`|The opacity of the layer (0.0 is fully transparent and 1.0 is fully opaque)|number|
|`format`|Either "jpg" or "png"|string|

## RGB composites

The following illustrates the use of parameters to style a Landsat 8 image as a false-color composite:

In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
image_viz_params = {
    'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]
}

# Create an RGB image based on the visualization parameters.
image_viz = image.visualize(**image_viz_params)

# Display to folium map. 
m = folium.Map(location=[37.5010, -122.1899], zoom_start=10, height=500)
m.add_ee_layer(image_viz, None, 'false color composite')
display(m)

In this example, band `'B5'` is assigned to red, `'B4'` is assigned to green, and `'B3'` is assigned to blue.

## Color palettes

To display a single band of an image in color, set the `palette` parameter with a color ramp represented by a list of CSS-style color strings. (See [this reference](http://en.wikipedia.org/wiki/Web_colors) for more information). The following example illustrates how to use colors from cyan (`'00FFFF'`) to blue (`'0000FF'`) to render a [Normalized Difference Water Index (NDWI)](http://www.tandfonline.com/doi/abs/10.1080/01431169608948714) image:

In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters.
ndwi = image.normalizedDifference(['B3', 'B5'])

# Define the visualization parameters.
ndwi_viz_params = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}

# Create an RGB image based on the visualization parameters.
ndwi_viz = ndwi.visualize(**ndwi_viz_params)

# Display to folium map.
m = folium.Map(location=[37.5010, -122.1899], zoom_start=10, height=500)
m.add_ee_layer(ndwi_viz, None, 'NDWI')
display(m)

In this example, note that the `min` and `max` parameters indicate the range of pixel values to which the palette should be applied. Intermediate values are linearly stretched.

## Masking

You can use `image.updateMask()` to set the opacity of individual pixels based on where pixels in a mask image are non-zero. Pixels equal to zero in the mask are excluded from computations and the opacity is set to 0 for display. The following example uses an NDWI threshold (see the [Relational Operations section](https://developers.google.com/earth-engine/image_relational) for information on thresholds) to update the mask on the NDWI layer created previously:

In [ ]:
# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwi_viz_masked = ndwi_viz.updateMask(ndwi.gte(0.4))

# Display to folium map.
m = folium.Map(location=[37.5010, -122.1899], zoom_start=10, height=500)
m.add_ee_layer(ndwi_viz_masked, None, 'NDWI')
display(m)

## Mosaicking

You can use masking and `imageCollection.mosaic()` (see the [Mosaicking section](https://developers.google.com/earth-engine/ic_composite_mosaic) for information on mosaicking) to achieve various cartographic effects. The `mosaic()` method renders layers in the output image according to their order in the input collection. The following example uses `mosaic()` to combine the masked NDWI and the false color composite and obtain a new visualization:

In [ ]:
# Mosaic the visualization layers and display (or export).
mosaic = ee.ImageCollection([image_rgb, ndwi_rgb]).mosaic()

# Display to folium map.
m = folium.Map(location=[37.5010, -122.1899], zoom_start=10, height=500)
m.add_ee_layer(mosaic, None, 'NDWI')
display(m)

In this example, observe that a list of the two visualization images is provided to the `ImageCollection` constructor. The order of the list determines the order in which the images are rendered on the map.

## Clipping

The `image.clip()` method is useful for achieving cartographic effects. The following example clips the mosaic above to an arbitrary buffer zone around the city of San Francisco:

In [ ]:
# Create a circle by drawing a 20000 meter buffer around a point.
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)
mosaic_clipped = mosaic.clip(roi)

# Display to folium map.
m = folium.Map(location=[37.7599, -122.4481], zoom_start=10, height=500)
m.add_ee_layer(mosaic_clipped, None, 'NDWI masked')
display(m)

In the previous example, note that the coordinates are provided to the `Geometry` constructor and the buffer length is specified as 20,000 meters. Learn more about geometries on the [Geometries page](https://developers.google.com/earth-engine/geometries).

## Rendering categorical maps

Palettes are also useful for rendering discrete valued maps, for example a land cover map. In the case of multiple classes, use the palette to supply a different color for each class. (The `image.remap()` method may be useful in this context, to convert arbitrary labels to consecutive integers). The following example uses a palette to render land cover categories:

In [ ]:
# Load 2012 MODIS land cover and select the IGBP classification.
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define a palette for the 18 distinct land cover classes.
igbp_palette = [
    'aec3d4',  # water
    '152106',
    '225129',
    '369b47',
    '30eb5b',
    '387242',  # forest
    '6a2325',
    'c3aa69',
    'b76031',
    'd9903d',
    '91af40',  # shrub, grass
    '111149',  # wetlands
    'cdb33b',  # croplands
    'cc0013',  # urban
    '33280d',  # crop mosaic
    'd7cdcc',  # snow and ice
    'f7e084',  # barren
    '6f6f6f'   # tundra
]

# Specify the min and max labels and the color palette matching the labels.
cover_viz_params = {'min': 0, 'max': 17, 'palette': igbp_palette}

# Create an RGB image based on the visualization parameters.
cover_viz = cover.visualize(**cover_viz_params)

# Display to folium map.
m = folium.Map(location=[38, -95], zoom_start=4, height=500)
m.add_ee_layer(cover, None, 'IGBP classification')
display(m)

## Styled Layer Descriptors

You can use a Styled Layer Descriptor ([SLD](http://www.opengeospatial.org/standards/sld)) to render imagery for display. Provide `image.sldStyle()` with an XML description of the symbolization and coloring of the image, specifically the `RasterSymbolizer` element. Learn more about the `RasterSymbolizer` element [here](http://docs.geoserver.org/stable/en/user/styling/sld/reference/rastersymbolizer.html). For example, to render the land cover map described in the Rendering categorical maps section with an SLD, use:

In [ ]:
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define an SLD style of discrete intervals to apply to the image.
sld_intervals = """
<RasterSymbolizer>
  <ColorMap  type="intervals" extended="false" >
    <ColorMapEntry color="#aec3d4" quantity="0" label="Water"/>
    <ColorMapEntry color="#152106" quantity="1" label="Evergreen Needleleaf Forest"/>
    <ColorMapEntry color="#225129" quantity="2" label="Evergreen Broadleaf Forest"/>
    <ColorMapEntry color="#369b47" quantity="3" label="Deciduous Needleleaf Forest"/>
    <ColorMapEntry color="#30eb5b" quantity="4" label="Deciduous Broadleaf Forest"/>
    <ColorMapEntry color="#387242" quantity="5" label="Mixed Deciduous Forest"/>
    <ColorMapEntry color="#6a2325" quantity="6" label="Closed Shrubland"/>
    <ColorMapEntry color="#c3aa69" quantity="7" label="Open Shrubland"/>
    <ColorMapEntry color="#b76031" quantity="8" label="Woody Savanna"/>
    <ColorMapEntry color="#d9903d" quantity="9" label="Savanna"/>
    <ColorMapEntry color="#91af40" quantity="10" label="Grassland"/>
    <ColorMapEntry color="#111149" quantity="11" label="Permanent Wetland"/>
    <ColorMapEntry color="#cdb33b" quantity="12" label="Cropland"/>
    <ColorMapEntry color="#cc0013" quantity="13" label="Urban"/>
    <ColorMapEntry color="#33280d" quantity="14" label="Crop, Natural Veg. Mosaic"/>
    <ColorMapEntry color="#d7cdcc" quantity="15" label="Permanent Snow, Ice"/>
    <ColorMapEntry color="#f7e084" quantity="16" label="Barren, Desert"/>
    <ColorMapEntry color="#6f6f6f" quantity="17" label="Tundra"/>
  </ColorMap>
</RasterSymbolizer>"""

# Apply the SLD style to the image.
cover_sld = cover.sldStyle(sld_intervals)

# Display to folium map.
m = folium.Map(location=[38, -95], zoom_start=4, height=500)
m.add_ee_layer(cover_sld, None, 'IGBP classification styled')
display(m)

To create a visualization image with a color ramp, set the type of the `ColorMap` to 'ramp'. The following example compares the 'interval' and 'ramp' types for rendering a DEM:

In [ ]:
# Load SRTM Digital Elevation Model data.
image = ee.Image('CGIAR/SRTM90_V4')

# Define an SLD style of discrete intervals to apply to the image.
sld_intervals = """
    <RasterSymbolizer>
      <ColorMap type="intervals" extended="false" >
        <ColorMapEntry color="#0000ff" quantity="0" label="0"/>
        <ColorMapEntry color="#00ff00" quantity="100" label="1-100" />
        <ColorMapEntry color="#007f30" quantity="200" label="110-200" />
        <ColorMapEntry color="#30b855" quantity="300" label="210-300" />
        <ColorMapEntry color="#ff0000" quantity="400" label="310-400" />
        <ColorMapEntry color="#ffff00" quantity="1000" label="410-1000" />
      </ColorMap>
    </RasterSymbolizer>"""

# Define an sld style color ramp to apply to the image.
sld_ramp = """
    <RasterSymbolizer>
      <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#0000ff" quantity="0" label="0"/>
        <ColorMapEntry color="#00ff00" quantity="100" label="100" />
        <ColorMapEntry color="#007f30" quantity="200" label="200" />
        <ColorMapEntry color="#30b855" quantity="300" label="300" />
        <ColorMapEntry color="#ff0000" quantity="400" label="400" />
        <ColorMapEntry color="#ffff00" quantity="500" label="500" />
      </ColorMap>
    </RasterSymbolizer>"""

# Apply the SLD styles to the image.
intervals = image.sldStyle(sld_intervals)
ramp = image.sldStyle(sld_ramp)

# Display to folium map.
m = folium.Map(location=[42.0289, -76.8054], zoom_start=8, height=500)
m.add_ee_layer(intervals, None, 'SLD intervals')
m.add_ee_layer(ramp, None, 'SLD ramp')
map.add_child(folium.LayerControl())
display(m)

SLDs are also useful for stretching pixel values to improve visualizations of continuous data. For example, the following code compares the results of an arbitrary linear stretch with a min-max 'Normalization' and a 'Histogram' equalization:

In [ ]:
# Load a Landsat 8 raw image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Define a RasterSymbolizer element with '_enhance_' for a placeholder.
template_sld = """
<RasterSymbolizer>
  <ContrastEnhancement><_enhance_/></ContrastEnhancement>
  <ChannelSelection>
    <RedChannel>
      <SourceChannelName>B5</SourceChannelName>
    </RedChannel>
    <GreenChannel>
      <SourceChannelName>B4</SourceChannelName>
    </GreenChannel>
    <BlueChannel>
      <SourceChannelName>B3</SourceChannelName>
    </BlueChannel>
  </ChannelSelection>
</RasterSymbolizer>"""

# Get SLDs with different enhancements.
equalize_sld = template_sld.replace('_enhance_', 'Histogram')
normalize_sld = template_sld.replace('_enhance_', 'Normalize')

# Apply the SLD styles to the image.
equalized = image.sldStyle(equalize_sld)
normalized = image.sldStyle(normalize_sld)

# Apply a linear stretch to the image.
linear = image.visualize(**{
    'bands': ['B5', 'B4', 'B3'], 
    'min': 0,
    'max': 15000
})

# Display layers to folium map.
m = folium.Map(location=[37.5174, -122.0813], zoom_start=10, height=500)
m.add_ee_layer(linear, None, 'Linear')
m.add_ee_layer(equalized, None, 'Equalized')
m.add_ee_layer(normalized, None, 'Normalized')
m.add_child(folium.LayerControl())
display(m)

Points of note in reference to using SLDs in Earth Engine:

- OGC SLD 1.0 and OGC SE 1.1 are supported.
- The XML document passed in can be complete, or just the RasterSymbolizer element and down.
- Bands may be selected by their Earth Engine names or index ('1', '2', ...).
- The Histogram and Normalize contrast stretch mechanisms are not supported for floating point imagery.
- Opacity is only taken into account when it is 0.0 (transparent). Non-zero opacity values are treated as completely opaque.
- The OverlapBehavior definition is currently ignored.
- The ShadedRelief mechanism is not currently supported.
- The ImageOutline mechanism is not currently supported.
- The Geometry element is ignored.
- The output image will have histogram_bandname metadata if histogram equalization or normalization is requested.

## Thumbnail images

Use the `ee.Image.getThumbURL()` method to generate a PNG or JPEG thumbnail image for an `ee.Image` object. Printing the outcome of an expression ending with a call to `getThumbURL()` results in a URL being printed to the console. Visiting the URL sets Earth Engine servers to work on generating the requested thumbnail on-the-fly. The image is displayed in the browser when processing completes. It can be downloaded by selecting appropriate options from the image’s right-click context menu.

_**Note:** The authorization token to process the thumbnail lasts 2 hours. Until it expires, anyone with the authorization token can generate the image._

![](https://developers.google.com/earth-engine/images/Images_thumbnail.png)
SRTM digital elevation model displayed as a PNG thumbnail in the browser.

The `getThumbURL()` method shares parameters with `Map.addLayer()`, described in the visualization parameters table above. Additionally, it takes optional `dimension`, `region`, `crs`, and `format` arguments.

| Parameter	 | Description                                    | Type
|-- |-- |--
| dimension	 | Thumbnail dimensions in pixel units. If a<br>single integer is provided, it defines the<br>size of the image's larger aspect dimension<br>and scales the smaller dimension proportionally.<br>Defaults to 512 pixels for the larger image<br>aspect dimension. | A single integer or string<br>in the format: 'WIDTHxHEIGHT'
| region	 | The geospatial region of the image to render.<br>The whole image by default, or the bounds of a<br>provided geometry. | GeoJSON or a 2-D list<br>of at least three point<br>coordinates that define a<br>linear ring
| crs	     | The target projection e.g. 'EPSG:3857'.<br>Defaults to WGS84 ('EPSG:4326'). | String
| format	 | Defines thumbnail format as either PNG or JPEG.<br>The default PNG format is implemented as RGBA,<br>where the alpha channel represents valid and<br>invalid pixels, defined by the image’s mask().<br>Invalid pixels are transparent. The optional<br>JPEG format is implemented as RGB, where invalid<br>image pixels are zero filled across RGB channels. | String; either 'png' or 'jpg'

_**Caution:** The 'WIDTHxHEIGHT' dimension argument can alter the original aspect ratio of the data or region extent._

A single-band image will default to grayscale unless a `palette` argument is supplied. A multi-band image will default to RGB visualization of the first three bands, unless a `bands` argument is supplied. If only two bands are provided, the first band will map to red, the second to blue, and the green channel will be zero filled.

The following are a series of examples demonstrating various combinations of `getThumbURL()` parameter arguments. Visit the URLs printed to the console when you run this script to view the thumbnails.

In [ ]:
# Fetch a digital elevation model.
image = ee.Image('CGIAR/SRTM90_V4')

# Request a default thumbnail of the DEM with defined linear stretch.
# Set masked pixels (ocean) to 1000 so they map as gray.
thumbnail_1 = image.unmask(1000).getThumbURL({
    'min': 0,
    'max': 3000,
    'dimensions': 500,
})
print('Default extent:', thumbnail_1)

# Specify region by rectangle, define palette, set larger aspect dimension size.
thumbnail_2 = image.getThumbURL({
    'min': 0,
    'max': 3000,
    'palette': [
        '00A600', '63C600', 'E6E600', 'E9BD3A', 'ECB176', 'EFC2B3', 'F2F2F2'
    ],
    'dimensions': 500,
    'region': ee.Geometry.Rectangle([-84.6, -55.9, -32.9, 15.7]),
})
print('Rectangle region and palette:', thumbnail_2)

# Specify region by a linear ring and set display CRS as Web Mercator.
thumbnail_3 = image.getThumbURL({
    'min': 0,
    'max': 3000,
    'palette': [
        '00A600', '63C600', 'E6E600', 'E9BD3A', 'ECB176', 'EFC2B3', 'F2F2F2'
    ],
    'region':
        ee.Geometry.LinearRing([[-84.6, 15.7], [-84.6, -55.9], [-32.9, -55.9]]),
    'dimensions': 500,
    'crs': 'EPSG:3857'
})
print('Linear ring region and specified crs:', thumbnail_3)

_**Note**:_

- _Thumbnail images are also available as UI elements. See [here](https://developers.google.com/earth-engine/ui_widgets#uithumbnail) for more details._
- _`getThumbURL` is intended as a method for producing preview images you might include in presentations, websites, and social media posts. Its size limitation is 100,000,000 pixels and the browser can timeout for complicated requests. If you want a large image or have a complex process, see the [Exporting Data](https://developers.google.com/earth-engine/exporting) page._